# Lista segmentada de clientes

O objetivo deste exercício é segmentar um conjunto de clientes do varejo baseado em suas trasações. Uma das técnicas mais utilizadas para segmentação é a análise [RFM](https://en.wikipedia.org/wiki/RFM_(market_research)), uma combinação de três métricas: 
* *Recency*, quão recentemente o cliente efetuou uma compra.
* *Frequency*, com que frequência o cliente compra.
* *Monetary value*, valor gasto nas compras do cliente.

Para cada uma das métricas agruparemos os clientes em quatro categorias utilizando *K-means clustering*. No final combinaremos os clusters para gerar nossa lista segmentada.


# Importação de pacotes


In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Preparação de dados

In [2]:
data = pd.read_csv('../data/OnlineRetail.csv',encoding="ISO-8859-1", header=0)

In [3]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
data.isnull().sum().sum()

136534

In [6]:
data.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

Precisamos converter a coluna InvoiceDate para representar uma data, assim podemos calcular a próxima métrica. Também retiramos os valores nulos.

In [7]:
data = data.dropna()
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [8]:
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

# Ordenação dos clusters

Para métrica *Recency*, valores menores são preferíveis. Já para *Frequency* e *Monetary Value*, quanto maior melhor. Levando isso em consideração, utilizaremos a seguinte função auxiliar para ordenar os clusters de acordo com a preferência dos valores das métricas (ascendentes ou descendentes). Isso facilitará quando formos combinar as métricas.

In [9]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

# Recency

Primeiramente vamos gerar a métrica *Recency* para depois agrupar os clientes com base nela (*clustering*).

In [10]:
costumers = pd.DataFrame(data['CustomerID'].unique())
costumers.columns = ['CustomerID']

#Dataframe com a data da compra mais recente de cada cliente e a sua diferença pra mais recente de todas as compras.
purchase_date = data.groupby('CustomerID').InvoiceDate.max().reset_index()
purchase_date.columns = ['CustomerID','Latest']
purchase_date['R'] = (purchase_date['Latest'].max() - purchase_date['Latest']).dt.days

#Uniao dos dataframes
costumers = pd.merge(costumers, purchase_date[['CustomerID','R']], on='CustomerID')

costumers.head()


,CustomerID,R
0,17850.0,69
1,13047.0,47
2,12583.0,0
3,13748.0,159
4,15100.0,70


Agora treinaremos um estimador K-means para determinar em qual cluster cada cliente estará de acordo com sua última compra. Além de ordenar os índices dos clusters de maneira descendente. 

In [11]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(costumers[['R']])
# Cluster R
costumers['RC'] = kmeans.predict(costumers[['R']])

costumers = order_cluster('RC', 'R',costumers,False)

costumers.groupby('RC')['R'].describe()

,count,mean,std,min,25%,50%,75%,max
RC,,,,,,,,
0,76.0,588.934211,94.109912,454.0,485.0,638.0,666.0,697.0
1,618.0,278.504854,44.639643,211.0,243.0,270.0,311.0,424.0
2,952.0,143.115546,35.023641,88.0,114.0,141.5,174.0,210.0
3,2726.0,32.363536,24.061944,0.0,12.0,25.0,51.0,87.0


# Frequency

Repetimos o mesmo processo para métrica de frequência, fazendo a união com o dataframe de clientes e ordenando de maneira ascendente.

In [12]:
frequency = data.groupby('CustomerID').InvoiceDate.count().reset_index()
frequency.columns = ['CustomerID','F']

costumers = pd.merge(costumers, frequency, on='CustomerID')


kmeans = KMeans(n_clusters=4)
kmeans.fit(costumers[['F']])
costumers['FC'] = kmeans.predict(costumers[['F']])

costumers = order_cluster('FC', 'F',costumers,True)

costumers.groupby('FC')['F'].describe()

,count,mean,std,min,25%,50%,75%,max
FC,,,,,,,,
0,3862.0,49.753755,44.812998,1.0,15.0,34.0,73.0,189.0
1,482.0,329.107884,132.670589,190.0,228.0,286.0,393.5,803.0
2,24.0,1349.750000,508.637759,872.0,1003.5,1156.0,1541.0,2782.0
3,4.0,5914.000000,1473.845537,4642.0,5006.5,5515.5,6423.0,7983.0


# Monetary Value

E finalmente geramos os clusters para a última métrica. 

In [13]:
# Nova coluna com o valor total de cada compra
data['M'] = data['UnitPrice'] * data['Quantity']
monetary = data.groupby('CustomerID').M.sum().reset_index()

costumers = pd.merge(costumers, monetary, on='CustomerID')

kmeans = KMeans(n_clusters=4)
kmeans.fit(costumers[['M']])
costumers['MC'] = kmeans.predict(costumers[['M']])


costumers = order_cluster('MC', 'M',costumers,True)

costumers.groupby('MC')['M'].describe()

,count,mean,std,min,25%,50%,75%,max
MC,,,,,,,,
0,4252.0,1160.746285,1403.441736,-4287.63,286.565,623.575,1476.0700,8257.04
1,102.0,15390.295980,7557.269118,8393.22,10192.235,12395.080,17460.8075,40340.78
2,15.0,71423.516000,28632.631870,50415.49,52287.280,57385.880,77008.7300,132572.62
3,3.0,241136.560000,47874.073443,187482.17,221960.330,256438.490,267963.7550,279489.02


# Métricas combinadas

Depois de determinar em que grupo cada cliente se encontra por métrica, vamos atribuí-los um *score* que combine as três métricas. Como ordenamos os índices de cada cluster de acordo com o que é melhor para cada métrica, se simplesmente somarmos o valor dos clusters podemos perceber que os valores menos significativos de todas métrica, **em média**, obterão scores mais baixos e os mais significativos obterão scores mais altos.

In [14]:
costumers['Score'] = costumers['RC'] + costumers['FC'] + costumers['MC']
costumers.groupby('Score')['R','F','M'].mean()

/Users/raonilourenco/miniconda2/envs/testenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,R,F,M
Score,,,
0,588.934211,19.118421,217.693816
1,278.649837,25.368078,376.201515
2,144.181230,40.111111,680.362168
3,36.289978,63.689532,1191.074866
4,21.931373,303.007353,4382.193186
5,13.949367,512.443038,14196.265949
6,19.909091,955.909091,44249.222727
7,9.333333,2432.555556,98056.746667
8,4.666667,4372.000000,156394.183333


# Segmentos

Para finalizar o exercício vamos dividir os clientes em três segmentos: **Baixo, médio e alto** valor de retorno para os negócios. Cada segmento será a união de cliente com três scores próximos. 

In [15]:
costumers['Segment'] = 'Low'
costumers.loc[costumers['Score']>2,'Segment'] = 'Medium' 
costumers.loc[costumers['Score']>4,'Segment'] = 'High' 

In [16]:
costumers.head()

,CustomerID,R,RC,F,FC,M,MC,Score,Segment
0,17850.0,69,3,312,1,5288.63,0,4,Medium
1,13047.0,47,3,196,1,3079.10,0,4,Medium
2,12583.0,0,3,251,1,7187.34,0,4,Medium
3,14688.0,0,3,359,1,5107.38,0,4,Medium
4,12431.0,2,3,240,1,6416.39,0,4,Medium


Percebe-se que o id co cliente está em formato float, trocaremos para inteiro.

In [17]:
costumers['CustomerID'] = costumers['CustomerID'].astype(int)

In [20]:
costumers.to_csv('../api/segments/costumers.csv')

# Metadados do dataset final

In [19]:
costumers.dtypes

CustomerID      int64
R               int64
RC              int64
F               int64
FC              int64
M             float64
MC              int64
Score           int64
Segment        object
dtype: object